In [1]:
!pip install torch seqeval datasets evaluate accelerate peft transformers==4.28.0 --quiet

In [4]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
key= user_secrets.get_secret("wandb")
wandb.login(key=key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
import torch
import pandas as pd
import pickle

In [6]:
input_path='/kaggle/Input/'
output_path='/kaggle/working/'

In [7]:
label_list= ['O', 'B-TSK','I-TSK','B-MTD','I-MTD','B-DST','I-DST']

# Dataset Preparation

## Load dataset

In [8]:
with open(input_path+'Data/df_scirex.pkl', 'rb') as file:
    data=pickle.load(file)
data

,sentences,tags
0,"[[document, :, Deep, Label, Distribution, Lear...","[[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, ..."
1,"[[In, this, work, we, explore, recent, advance...","[[O, O, O, O, O, O, O, O, B-MTD, I-MTD, I-MTD,..."
2,"[[Bridging, Saliency, Detection, to, Weakly, S...","[[B-TSK, I-TSK, I-TSK, O, B-TSK, I-TSK, I-TSK,..."
3,"[[document, :, The, IBM, 2016, English, Conver...","[[O, O, O, B-TSK, I-TSK, I-TSK, I-TSK, I-TSK, ..."
4,"[[document, :, Rethinking, the, Inception, Arc...","[[O, O, O, O, B-MTD, I-MTD, O, B-TSK, I-TSK], ..."
...,...,...
433,"[[document, :, Invertible, Conditional, GANs, ...","[[O, O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK],..."
434,"[[Asymmetric, Tri, -, training, for, Unsupervi...","[[B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK,..."
435,"[[A, Convolutional, Encoder, Model, for, Neura...","[[O, B-MTD, I-MTD, I-MTD, O, B-TSK, I-TSK, I-T..."
436,"[[document, :, Learning, to, Compare, :, Relat...","[[O, O, B-TSK, O, O, O, B-MTD, I-MTD, O, O, B-..."


In [9]:
data=data.iloc[0:1,:]
data

,sentences,tags
0,"[[document, :, Deep, Label, Distribution, Lear...","[[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, ..."


## Change the format

In [10]:
data=data.explode(['sentences','tags']).reset_index(drop=True)
data

,sentences,tags
0,"[document, :, Deep, Label, Distribution, Learn...","[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I..."
1,"[Convolutional, Neural, Networks, (, ConvNets,...","[B-MTD, I-MTD, I-MTD, I-MTD, I-MTD, I-MTD, O, ..."
2,"[A, large, labeled, training, set, is, one, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"[However, ,, it, is, difficult, to, collect, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Fortunately, ,, there, is, ambiguous, informa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
606,"[{, IEEEbiography}, [], Xin, Geng, (, M’13, ),...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
607,"[D, degree, from, Deakin, University, ,, Austr...","[O, O, O, O, O, O, O, O, O, O]"
608,"[He, joined, the, School, of, Computer, Scienc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
609,"[He, has, authored, over, 50, refereed, papers...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [11]:
data['ner_tags'] = pd.Series(dtype='object')
for i,item in enumerate(data.tags):
    ner_list=[]
    for tag in item:
        for j,label in enumerate(label_list):
            if tag==label:
                ner_list.append(j)
    data['ner_tags'][i]=ner_list
data

,sentences,tags,ner_tags
0,"[document, :, Deep, Label, Distribution, Learn...","[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I...","[0, 0, 3, 4, 4, 4, 0, 1, 2]"
1,"[Convolutional, Neural, Networks, (, ConvNets,...","[B-MTD, I-MTD, I-MTD, I-MTD, I-MTD, I-MTD, O, ...","[3, 4, 4, 4, 4, 4, 0, 0, 0, 1, 2, 0, 0, 1, 2, ..."
2,"[A, large, labeled, training, set, is, one, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[However, ,, it, is, difficult, to, collect, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Fortunately, ,, there, is, ambiguous, informa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
606,"[{, IEEEbiography}, [], Xin, Geng, (, M’13, ),...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
607,"[D, degree, from, Deakin, University, ,, Austr...","[O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
608,"[He, joined, the, School, of, Computer, Scienc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
609,"[He, has, authored, over, 50, refereed, papers...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
data['tokens'] = [i for i in data['sentences']]
data['id'] = [i for i in data.index]
data

,sentences,tags,ner_tags,tokens,id
0,"[document, :, Deep, Label, Distribution, Learn...","[O, O, B-MTD, I-MTD, I-MTD, I-MTD, O, B-TSK, I...","[0, 0, 3, 4, 4, 4, 0, 1, 2]","[document, :, Deep, Label, Distribution, Learn...",0
1,"[Convolutional, Neural, Networks, (, ConvNets,...","[B-MTD, I-MTD, I-MTD, I-MTD, I-MTD, I-MTD, O, ...","[3, 4, 4, 4, 4, 4, 0, 0, 0, 1, 2, 0, 0, 1, 2, ...","[Convolutional, Neural, Networks, (, ConvNets,...",1
2,"[A, large, labeled, training, set, is, one, of...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[A, large, labeled, training, set, is, one, of...",2
3,"[However, ,, it, is, difficult, to, collect, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[However, ,, it, is, difficult, to, collect, s...",3
4,"[Fortunately, ,, there, is, ambiguous, informa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Fortunately, ,, there, is, ambiguous, informa...",4
...,...,...,...,...,...
606,"[{, IEEEbiography}, [], Xin, Geng, (, M’13, ),...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[{, IEEEbiography}, [], Xin, Geng, (, M’13, ),...",606
607,"[D, degree, from, Deakin, University, ,, Austr...","[O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[D, degree, from, Deakin, University, ,, Austr...",607
608,"[He, joined, the, School, of, Computer, Scienc...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[He, joined, the, School, of, Computer, Scienc...",608
609,"[He, has, authored, over, 50, refereed, papers...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[He, has, authored, over, 50, refereed, papers...",609


In [13]:
from datasets import Dataset
dataset = Dataset.from_pandas(data[['id','tokens','ner_tags']])
dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 611
})

## Split dataset

In [14]:
# LOAD OR TRAIN MODEL
TRAIN = 1 # 1 to TRAIN WEIGHTS or 0 to LOAD WEIGHTS

# TRAIN/TEST SPLIT
TRAIN_TEST_SPLIT = 0.2
# TRAIN/VALIDATION SPLIT
TRAIN_VAL_SPLIT = 0.125
# Train:Test:Evaluation=7:2:1 (3.5:1:0.5)

# RANDOM SEED FOR REPRODUCIBILITY
RANDOM_SEED = 42

In [15]:
from sklearn.model_selection import train_test_split
train_test = dataset.train_test_split(test_size=TRAIN_TEST_SPLIT)
train_valid = train_test['train'].train_test_split(test_size=TRAIN_VAL_SPLIT)

In [16]:
from datasets import DatasetDict
datasets = DatasetDict({
    'train': train_test['train'],
    'test': train_test['test'],
    'valid': train_valid['test']})
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 488
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 123
    })
    valid: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 61
    })
})

# Train baselines on token classification

## Build model

In [19]:
MODELS={"checkpoint":["allenai/scibert_scivocab_uncased","malteos/scincl","allenai/specter2", "johngiorgi/declutr-sci-base"],
       "name":["SciBERT","SciNCL","SPECTER","DeCLTUR","BioBERT","SciFive"]}
# change id to train on different baseline models
id=0

In [ ]:
from transformers import AutoTokenizer
if id==3:
    tokenizer = AutoTokenizer.from_pretrained(MODELS["checkpoint"][id], add_prefix_space=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODELS["checkpoint"][id])

In [22]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True, max_length=512)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
def word_id_func(input_ids, print_labs=False):
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    word_ids = []
    i=0
    spec_toks = ['[CLS]', '[SEP]', '[PAD]']
    for t in tokens:
        if t in spec_toks:
            word_ids.append(-100)
            print(t, i) if print_labs else None
        elif t.startswith('▁'):
            i += 1
            word_ids.append(i)
            print(t, i) if print_labs else None
        else:
            word_ids.append(i)
            print(t, i) if print_labs else None
        print("Total:", i) if print_labs else None
    return word_ids

def tokenize_and_align_labels_roberta(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,is_split_into_words=True, max_length=512)
    labels = []
    word_ids_list = []
    for input_ids in tokenized_inputs["input_ids"]:
        wids = word_id_func(input_ids, print_labs=False)
        word_ids_list.append(wids)
    
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = word_ids_list[i]
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx == -100:
                label_ids.append(-100)
            #We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx-1])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx-1] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# tokenized_data= tokenize_and_align_labels(data)
if id==3:
    tokenized_datasets = datasets.map(tokenize_and_align_labels_roberta, batched=True)
else:
    tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## Training

In [25]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(device))
#     print(torch.cuda.current_device())
else:
    device ='cpu'
output_dir = output_path+'Models/cache'
# BATCH SIZE
# TRY 4, 8, 16, 32, 64, 128, 256. REDUCE IF OOM ERROR, HIGHER FOR TPUS
BATCH_SIZES = 4
# EPOCHS - TRANSFORMERS ARE TYPICALLY FINE-TUNED BETWEEN 1 AND 3 EPOCHS 
EPOCHS = 10
# RANDOM SEED FOR REPRODUCIBILITY
RANDOM_SEED = 42
# SPECIFY THE WEIGHTS AND BIASES PROJECT NAME
%env WANDB_PROJECT = Thesis
# DETERMINE WHETHER TO SAVE THE MODEL IN THE 100GB OF FREE W&B STORAGE
%env WANDB_LOG_MODEL = True

env: WANDB_PROJECT=Thesis
env: WANDB_LOG_MODEL=True


In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(MODELS["checkpoint"][id], num_labels=len(label_list))

In [27]:
# Training schedule
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers.optimization import Adafactor, AdafactorSchedule
learning_rate = 0.0000075
lr_max = learning_rate * BATCH_SIZES
weight_decay = 0.05
print("The maximum learning rate is: ",lr_max)
num_train_samples = len(datasets["train"])
warmup_ratio = 0.2 # Percentage of total steps to go from zero to max learning rate
num_cycles=0.8 # The cosine exponential rate
num_training_steps = num_train_samples*EPOCHS/BATCH_SIZES
num_warmup_steps = num_training_steps*warmup_ratio
#Optimizer
optimizer = torch.optim.AdamW(
                model.parameters(),
                lr=lr_max,
                weight_decay=weight_decay
            )
# Learning Rate Schedule
lr_sched = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                   num_warmup_steps=num_warmup_steps,
                                   num_training_steps = num_training_steps,
                                   num_cycles=num_cycles)

The maximum learning rate is:  3e-05


In [28]:
# Logging date for w&b
from datetime import date
today = date.today()
log_date = today.strftime("%d-%m-%Y")

In [29]:
from transformers import TrainingArguments
args = TrainingArguments(output_dir = output_dir,
                         evaluation_strategy = "epoch",
                         learning_rate=lr_max,
                         per_device_train_batch_size=BATCH_SIZES,
                         per_device_eval_batch_size=BATCH_SIZES,
                         num_train_epochs=EPOCHS,
                         weight_decay=weight_decay,
                         lr_scheduler_type = 'cosine',
                         warmup_ratio=warmup_ratio,
                         logging_strategy="epoch",
                         save_strategy="epoch",
                         seed=RANDOM_SEED,
                         report_to = 'wandb', # enable logging to W&B
                         run_name = MODELS["name"][id] +"-"+log_date,
                         metric_for_best_model="f1",
                         load_best_model_at_end = True,
                        )

In [30]:
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    print(predictions,len(predictions))
    print(labels,len(labels))
    predictions = np.argmax(predictions, axis=2)
    print(predictions,type(predictions))
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]
    print(type(true_predictions),len(true_predictions))
    print(type(true_labels),len(true_labels))
    # Define the metric parameters
    overall_precision = precision_score(true_labels, true_predictions, zero_division=1)
    overall_recall = recall_score(true_labels, true_predictions, zero_division=1)
    overall_f1 = f1_score(true_labels, true_predictions, zero_division=1)
    overall_accuracy = accuracy_score(true_labels, true_predictions)
    # Return a dictionary with the calculated metrics
    return {
        "precision": overall_precision,
        "recall": overall_recall,
        "f1": overall_f1,
        "accuracy": overall_accuracy,}

In [31]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [32]:
from transformers import Trainer, TrainerCallback
class EmptyCacheCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        torch.cuda.empty_cache()
empty_cache_callback = EmptyCacheCallback()

trainer = Trainer(
                model=model,
                args=args,
                train_dataset=tokenized_datasets["train"],
                eval_dataset=tokenized_datasets["test"],
                data_collator=data_collator,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
                optimizers=(optimizer, lr_sched),
                callbacks=[empty_cache_callback],
                )

In [ ]:
trainer.train()

In [ ]:
# Evaluate based on the chosen epoch (usually best or last)
trainer.evaluate()

In [ ]:
trainer.save_model(output_path+"Models/"+MODELS["name"][id])

In [ ]:
wandb.finish()

## Predictions on validation dataset

In [ ]:
# Load the model from the checkpoint
loaded_model = AutoModelForTokenClassification.from_pretrained(output_path+"Models/"+MODELS["name"][id])

In [ ]:
pred_trainer = Trainer(
    loaded_model,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# classification report
predictions, labels, _ = pred_trainer.predict(tokenized_datasets["valid"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
    ]

# Generate the metrics and display
results = classification_report(true_labels, true_predictions, zero_division=1)
print(results)

In [ ]:
check=123
datasets["valid"][check]

In [ ]:
# Have a look at the predicted extracted data
check_pred = zip(datasets["valid"][check]['tokens'], true_predictions[check])
for tup in check_pred:
    if tup[1] != 'O':
        print(tup)

In [ ]:
# Compare to the actual labels
check_true = zip(datasets["valid"][check]['tokens'], true_labels[check])
for tup in check_true:
    if tup[1] != 'O':
        print(tup)